In [ ]:
token ='insert your neuprint token here'

from neuprint import Client, fetch_adjacencies, merge_neuron_properties, fetch_neurons, NeuronCriteria as NC, fetch_synapses, SynapseCriteria as SC, skeleton_segments, fetch_synapse_connections
from neuprint.utils import connection_table_to_matrix
c = Client('neuprint.janelia.org', dataset='manc:v1.2.1', token=token)
c.fetch_version()

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
from matplotlib.colors import LinearSegmentedColormap, ListedColormap

fig_path = 'C:/Users/LAB-JJ/Desktop/code/figs/'
plt.rcParams.update(plt.rcParamsDefault)
plt.rcParams['pdf.fonttype']=42
plt.rcParams['svg.fonttype'] = 'none'
plt.rcParams["font.family"] = "Arial"
plt.rcParams["font.size"] = 11

In [ ]:
md_4_neurons = [15263,
 18951,
 21105,
 21209,
 21219,
 21773,
 21976,
 22045,
 22090,
 22123,
 22217,
 22228,
 22247,
 22411,
 22543,
 22613,
 22719,
 22995,
 23476,
 23532,
 23783,
 24165,
 24180,
 25494,
 25636,
 26664,
 101527,
 152738,
 154741,
 163802]
post_criteria = NC(bodyId=md_4_neurons)

#Fetch all direct connections between a set of upstream neurons
neuron_df, conn_df = fetch_adjacencies(None, post_criteria)
presynatpic_neurons = conn_df[conn_df.weight>=1].bodyId_pre.unique()
pre_df_info, pre_conn_info = fetch_neurons(NC(bodyId=presynatpic_neurons, status='Traced'))
conn_df_filt_MERGED = conn_df.merge(pre_df_info, how='left', left_on='bodyId_pre', right_on='bodyId')
#change "sensory ascending' to 'sensory neuron'
conn_df_filt_MERGED['class'] = conn_df_filt_MERGED['class'].replace({'sensory ascending': 'sensory neuron', 'efferent ascending': 'efferent neuron'})
conn_df_filt_MERGED = conn_df_filt_MERGED[~conn_df_filt_MERGED['class'].str.contains('TBD', na=True)]

In [ ]:
def plot_stacked_class_barplot(data , columnId=None, columnInstance=None):
    ntCount = data.groupby([columnId,'class'])['weight'].sum().reset_index(name='classCount')
    ntCount_tot = data.groupby([columnId])['weight'].sum().reset_index(name='classCount_total')

    merged_df = pd.merge(ntCount, ntCount_tot, on=columnId, suffixes=('_individual', '_total'))

    # Step 2: Calculate percentages
    merged_df['proportion'] = (merged_df['classCount'] / merged_df['classCount_total'])

    # Step 3: Pivot the DataFrame for plotting
    pivot_df = merged_df.pivot_table(index=columnId, columns='class', values='proportion', fill_value=0)
    #Ids = data.sort_values(columnInstance, ascending=True).bodyId_pre.unique()
    
    return pivot_df# Ids



In [ ]:
ntCount = conn_df_filt_MERGED.groupby(['bodyId_post','predictedNt'])['predictedNt'].count().reset_index(name='ntCount')
ntCount_tot = conn_df_filt_MERGED.groupby(['bodyId_post'])['predictedNt'].count().reset_index(name='ntCount_total')

merged_df = pd.merge(ntCount, ntCount_tot, on='bodyId_post', suffixes=('_individual', '_total'))

# Step 2: Calculate percentages
merged_df['proportion'] = (merged_df['ntCount'] / merged_df['ntCount_total'])

# Step 3: Pivot the DataFrame for plotting
pivot_df = merged_df.pivot_table(index='bodyId_post', columns='predictedNt', values='proportion', fill_value=0)
post = [ 15263,
 18951,
 21105,
 21209,
 21219,
 21773,
 21976,
 22045,
 22090,
 22123,
 22217,
 22228,
 22247,
 22411,
 22543,
 22613,
 22719,
 22995,
 23476,
 23532,
 23783,
 24165,
 24180,
 25494,
 25636,
 26664,
 101527,
 152738,
 154741,
 163802]

# Step 4: Create the stacked bar plot
colors = ['#b0c2b5', '#3e7d50', '#074518']
cmap = ListedColormap(colors)


fig, ax = plt.subplots(figsize=(4.5, 1))

# Plot the data
pivot_df.reindex(post).plot(kind='bar', stacked=True, colormap=cmap, ax=ax)

# Set plot labels and title
ax.set_xlabel('md-A neuron')
ax.set_ylabel('Proportion')
#ax.set_title('Proportion of Each predictedNt per bodyId_pre')
ax.legend(title='neurotransmitter', bbox_to_anchor=(1.05, 1), loc='upper left')

plt.tight_layout()
plt.show()

In [ ]:
# declaring data
data = [(5.384886+2.578947), 2, (1.236181+3.071429), 1.417582,1.636364]
keys = ['sensory', 'efferent/motor', 'intrinsic', 'ascending', 'descending']
  
# declaring exploding pie
explode = [ 0.05, 0, 0, 0, 0]
# define Seaborn color palette to use
palette_color = sns.color_palette('Blues')
fig = plt.figure(figsize=(2.5,2.5))  
# plotting data on chart
plt.pie(data, labels=keys, colors=palette_color,
        explode=explode, autopct='%.0f%%')
  
# displaying chart
plt.show()

In [ ]:
# declaring data
data = [(6128+49), 2, (246+43), 129,18]
keys = ['sensory', 'efferent/motor', 'intrinsic', 'ascending', 'descending']
  
# declaring exploding pie
explode = [ 0.05, 0, 0, 0, 0]
# define Seaborn color palette to use
palette_color = sns.color_palette('Blues')
fig = plt.figure(figsize=(2.5,2.5))  
# plotting data on chart
plt.pie(data, labels=keys, colors=palette_color,
        explode=explode, autopct='%.0f%%')
  
# displaying chart
plt.show()

In [ ]:

# declaring data
data = [409, 2, 118, 31,6]
keys = ['sensory', 'efferent/motor', 'intrinsic', 'ascending', 'descending']
  
# declaring exploding pie
explode = [ 0.05, 0, 0, 0, 0]
# define Seaborn color palette to use
palette_color = sns.color_palette('Blues')
fig = plt.figure(figsize=(2.5,2.5))  
# plotting data on chart
plt.pie(data, labels=keys, colors=palette_color,
        explode=explode, autopct='%.0f%%')
  
# displaying chart
plt.show()

In [ ]:
abdominal_bristles = [11317,      11762,      12286,      12729,      12768,
            13825,      13860,      13877,      13954,      13967,
            14439,      15053,      15375,      15491,      15545,
            15570,      16049,      16161,      16279,      16291,
            16314,      16807,      16959,      17152,      17369,
            17446,      17478,      17504,      17749,      17937,
            18025,      18065,      18100,      18266,      18843,
            18986,      18993,      19170,      19177,      19210,
            19290,      19313,      19328,      19395,      19439,
            19531,      19650,      19692,      19695,      19832,
            19917,      19972,      19997,      20168,      20193,
            20315,      20327,      20389,      20398,      20606,
            20764,      20783,      20887,      20928,      20934,
            20975,      21016,      21019,      21143,      21347,
            21419,      21831,      21902,      22041,      22077,
            22107,      22141,      22497,      22628,      22796,
            22810,      22833,      22873,      23420,      24357,
            24575,      24680,      24913,      25107,      25218,
            25225,      25291,      25399,      25511,      25524,
            25631,      25649,      25748,      25929,      26009,
            26243,      26351,      26406,      26623,      26814,
            26895,      27223,      27423,      27433,      27454,
            27883,      27947,      28100,      28152,      28164,
            28166,      28345,      28369,      28379,      28427,
            28549,      28797,      28818,      29015,      29463,
            29558,      29622,      29625,      29919,      30743,
            30811,      31467,      31760,      32218,      32266,
            32404,      32429,      32444,      32553,      32722,
            32865,      32964,      33172,      33273,      33358,
            33376,      34064,      34101,      34684,      34802,
            34881,      35042,      35153,      35233,      35667,
            36120,      36744,      37016,      37042,      37364,
            37426,      38125,      38536,      38711,      38857,
            39298,      39360,      39950,      40068,      40091,
            40549,      40597,      40969,      41140,      41171,
            41210,      41211,      41276,      41471,      41665,
            41856,      42043,      42152,      42572,      42869,
            44053,      44093,      44140,      44252,      44442,
            44950,      44992,      45719,      45741,      45802,
            45909,      46003,      46251,      46584,      46957,
            47023,      47410,      48019,      48059,      48548,
            48946,      57496,      91674,      99990,     101454,
           101724,     104312,     104495,     152512,     152736,
           154264,     154743,     155091,     155385,     157465,
           158494,     158898,     159173,     163803,     165292,
           170623,     172470,     172471,     172854,     172855,
           174468,     177520,     178012,     192509,     192552,
           197649,     200916,     201110,     204554, 9539181305]

# edit subclass in conn_df_filt_MERGED to 'abdominal_bristle_sensory' if bodyId_pre is in abdominal_bristles
conn_df_filt_MERGED['subclass'] = conn_df_filt_MERGED.apply(lambda row: 'putative ANm mechanosensory bristle' if row['bodyId_pre'] in abdominal_bristles else row['subclass'], axis=1)
#change any NaN subclass to 'unknown'
conn_df_filt_MERGED['subclass'] = conn_df_filt_MERGED['subclass'].fillna('unknown')

## make a pie chart of the above data: how many sensory neurons are classified as 'putative ANm mechanosensory bristle' in proportion to total sensory neurons

sensory_counts = conn_df_filt_MERGED[conn_df_filt_MERGED['class']=='sensory neuron'].groupby('subclass')['bodyId_pre'].nunique().reset_index(name='num_neurons')
data = sensory_counts['num_neurons'].tolist()
keys = sensory_counts['subclass'].tolist()

explode = [0.05] + [0]*(len(data)-1)
palette_color = sns.color_palette('Purples', len(data))
fig = plt.figure(figsize=(2.5,2.5))
plt.pie(data, labels=keys, colors=palette_color, explode=explode, autopct='%.0f%%')
plt.show()